In [1]:
import pandas as pd
import os
os.chdir('../../scripts')
from analysis_tools_cython import *

In [15]:
import pandas as pd
import numpy as np

data = pd.read_csv('../som_candidates_eleanor-lite-batch-analyse.txt')
paths = data.iloc[:, 0].to_list()

# Create an empty list to store results
all_above_threshold = []

for i in paths:
    lc, lc_info = import_lightcurve(i)
    results, arrays = processing(lc, lc_info=lc_info, method='median')
    results = results.split()

    # Extracting time and bkg columns from lc
    time = lc['TIME']
    bkg = lc['FLUX_BKG']

    # Find the index of the closest value
    index_of_closest_value = np.argmin(np.abs(bkg - float(results[3])))  # Use the correct index for time

    # Defining a mad and its threshold for the lightcurve
    mad = np.nanmedian(bkg - np.nanmedian(bkg))
    threshold = 3 * mad

    # Get indices around the closest index (two indices on each side)
    indices_around = np.arange(max(0, index_of_closest_value - 2), min(len(bkg), index_of_closest_value + 3))

    # Convert lc to a pandas DataFrame
    df = lc.to_pandas()

    # Extracting the interested columns using iloc
    interested_cols = df.iloc[indices_around, [df.columns.get_loc('TIME'), df.columns.get_loc('FLUX_BKG')]]

    # Applying the condition on FLUX_BKG
    condition = interested_cols['FLUX_BKG'] > threshold

    # Check if all points meet the condition
    if condition.all():
        # Add relevant information to the list
        all_above_threshold.append({
            'Path': i,
            'Index of closest value': index_of_closest_value,
            'Closest value': bkg[index_of_closest_value],
            'Interested columns': interested_cols.to_dict(orient='list')
        })

# Print the list of results
print(all_above_threshold)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

